# 일별 뉴스 흐름 Feature 생성

이 노트북은 기사 단위 데이터(bitcoin_news_merged_0.csv)를 일별로 집계하여
일별 뉴스 흐름 feature 테이블(features_daily.csv)을 생성합니다.

## 주요 작업
1. 데이터 로드 및 전처리
2. 일별 기사량 및 톤 집계
3. 테마(themes) 분리 및 카운팅
4. 최종 데이터 병합 및 저장

In [1]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료")
print(f"pandas 버전: {pd.__version__}")
print(f"numpy 버전: {np.__version__}")

라이브러리 로드 완료
pandas 버전: 3.0.0
numpy 버전: 2.4.1


## 1. 데이터 로드 및 전처리

In [2]:
def load_and_preprocess_data(filepath):
    """
    기사 데이터를 로드하고 기본 전처리를 수행합니다.
    
    Parameters:
    -----------
    filepath : str
        입력 CSV 파일 경로
    
    Returns:
    --------
    df : pd.DataFrame
        전처리된 데이터프레임
    
    처리 내용:
    - date를 문자열로 유지
    - tone을 numeric으로 변환 (avg_tone 컬럼 사용)
    - themes(v2_themes) 결측치 제거
    - url 기준 중복 제거
    """
    print(f"\n{'='*60}")
    print("데이터 로드 시작...")
    print(f"{'='*60}")
    
    # CSV 파일 로드
    df = pd.read_csv(filepath, encoding='utf-8-sig')
    print(f"✓ 원본 데이터 로드 완료: {len(df):,}행")
    
    # 컬럼명 확인 및 매핑
    print(f"\n원본 컬럼: {list(df.columns)}")
    
    # 컬럼명 표준화 (avg_tone -> tone, v2_themes -> themes)
    df = df.rename(columns={
        'avg_tone': 'tone',
        'v2_themes': 'themes'
    })
    
    # date를 문자열로 변환 (이미 문자열이지만 명시적으로 변환)
    df['date'] = df['date'].astype(str)
    
    # tone을 numeric으로 변환 (변환 불가능한 값은 NaN)
    df['tone'] = pd.to_numeric(df['tone'], errors='coerce')
    print(f"✓ tone 컬럼 변환 완료 (결측치: {df['tone'].isna().sum()}개)")
    
    # themes 결측치 제거
    before_drop = len(df)
    df = df.dropna(subset=['themes'])
    print(f"✓ themes 결측치 제거: {before_drop - len(df):,}행 제거")
    
    # themes를 문자열로 변환
    df['themes'] = df['themes'].astype(str)
    
    # url 기준 중복 제거 (같은 기사가 여러 번 수집된 경우)
    before_dedup = len(df)
    df = df.drop_duplicates(subset=['url'], keep='first')
    print(f"✓ URL 중복 제거: {before_dedup - len(df):,}행 제거")
    
    print(f"\n최종 데이터: {len(df):,}행")
    print(f"날짜 범위: {df['date'].min()} ~ {df['date'].max()}")
    print(f"고유 날짜 수: {df['date'].nunique()}일")
    
    return df

In [3]:
# 데이터 로드 및 전처리 실행
input_file = './data/processed/bitcoin_news_merged_0.csv'
df = load_and_preprocess_data(input_file)

# 데이터 샘플 확인
print("\n[데이터 샘플]")
display(df.head(3))

print("\n[데이터 정보]")
print(df.info())


데이터 로드 시작...
✓ 원본 데이터 로드 완료: 8,304행

원본 컬럼: ['date', 'time', 'source', 'url', 'v2_themes', 'avg_tone', 'positive_score', 'negative_score', 'polarity']
✓ tone 컬럼 변환 완료 (결측치: 0개)
✓ themes 결측치 제거: 242행 제거
✓ URL 중복 제거: 0행 제거

최종 데이터: 8,062행
날짜 범위: 20250901 ~ 20251031
고유 날짜 수: 61일

[데이터 샘플]


,date,time,source,url,themes,tone,positive_score,negative_score,polarity
0,20250901,00:00,biztoc.com,https://biztoc.com/x/57a509968ffd0464,ECON_BITCOIN;,6.122449,6.122449,0.000000,6.122449
1,20250901,00:00,biztoc.com,https://biztoc.com/x/643e611a0b7be9d5,ECON_BITCOIN;TAX_FNCACT;TAX_FNCACT_ANALYST;TAX...,-1.785714,1.785714,3.571429,5.357143
2,20250901,00:00,deadline.com,https://deadline.com/2025/08/tuner-review-leo-...,TAX_FNCACT;TAX_FNCACT_DIRECTOR;TAX_FNCACT_MAN;...,1.187215,4.657534,3.470320,8.127854



[데이터 정보]
<class 'pandas.DataFrame'>
Index: 8062 entries, 0 to 8303
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            8062 non-null   str    
 1   time            8062 non-null   str    
 2   source          8062 non-null   str    
 3   url             8062 non-null   str    
 4   themes          8062 non-null   str    
 5   tone            8062 non-null   float64
 6   positive_score  8062 non-null   float64
 7   negative_score  8062 non-null   float64
 8   polarity        8062 non-null   float64
dtypes: float64(4), str(5)
memory usage: 629.8 KB
None


## 2. 일별 기사량 및 톤 집계

In [4]:
def aggregate_daily_stats(df):
    """
    일별 기사량, 소스 수, 톤 통계를 집계합니다.
    
    Parameters:
    -----------
    df : pd.DataFrame
        전처리된 기사 데이터
    
    Returns:
    --------
    daily_stats : pd.DataFrame
        일별 집계 결과 (date를 인덱스로 가짐)
    
    생성 컬럼:
    - n_articles: 해당 날짜의 기사 수
    - n_sources: 해당 날짜의 고유 source 개수
    - tone_mean: tone 평균
    - tone_std: tone 표준편차
    - tone_neg_share: tone < 0 인 기사 비율
    - tone_pos_share: tone > 0 인 기사 비율
    """
    print(f"\n{'='*60}")
    print("일별 통계 집계 시작...")
    print(f"{'='*60}")
    
    # 날짜별로 그룹화
    grouped = df.groupby('date')
    
    # 1) 기사 수 카운트
    n_articles = grouped.size()
    print(f"✓ 일별 기사 수 계산 완료")
    
    # 2) 고유 소스 수 계산
    n_sources = grouped['source'].nunique()
    print(f"✓ 일별 고유 소스 수 계산 완료")
    
    # 3) tone 평균
    tone_mean = grouped['tone'].mean()
    print(f"✓ tone 평균 계산 완료")
    
    # 4) tone 표준편차
    tone_std = grouped['tone'].std()
    print(f"✓ tone 표준편차 계산 완료")
    
    # 5) tone이 음수인 기사 비율
    # 각 날짜별로 tone < 0인 기사 수를 세고, 전체 기사 수로 나눔
    tone_neg_count = grouped.apply(lambda x: (x['tone'] < 0).sum())
    tone_neg_share = tone_neg_count / n_articles
    print(f"✓ 부정 톤 비율 계산 완료")
    
    # 6) tone이 양수인 기사 비율
    tone_pos_count = grouped.apply(lambda x: (x['tone'] > 0).sum())
    tone_pos_share = tone_pos_count / n_articles
    print(f"✓ 긍정 톤 비율 계산 완료")
    
    # 모든 통계를 하나의 DataFrame으로 결합
    daily_stats = pd.DataFrame({
        'n_articles': n_articles,
        'n_sources': n_sources,
        'tone_mean': tone_mean,
        'tone_std': tone_std,
        'tone_neg_share': tone_neg_share,
        'tone_pos_share': tone_pos_share
    })
    
    print(f"\n일별 통계 집계 완료: {len(daily_stats)}일")
    print(f"평균 일별 기사 수: {daily_stats['n_articles'].mean():.1f}")
    print(f"평균 일별 소스 수: {daily_stats['n_sources'].mean():.1f}")
    
    return daily_stats

In [5]:
# 일별 통계 집계 실행
daily_stats = aggregate_daily_stats(df)

# 결과 확인
print("\n[일별 통계 샘플]")
display(daily_stats.head(10))

print("\n[통계 요약]")
display(daily_stats.describe())


일별 통계 집계 시작...
✓ 일별 기사 수 계산 완료
✓ 일별 고유 소스 수 계산 완료
✓ tone 평균 계산 완료
✓ tone 표준편차 계산 완료
✓ 부정 톤 비율 계산 완료
✓ 긍정 톤 비율 계산 완료

일별 통계 집계 완료: 61일
평균 일별 기사 수: 132.2
평균 일별 소스 수: 80.9

[일별 통계 샘플]


,n_articles,n_sources,tone_mean,tone_std,tone_neg_share,tone_pos_share
date,,,,,,
20250901,101,50,1.108581,3.068520,0.405941,0.544554
20250902,157,79,0.679788,2.812515,0.248408,0.687898
20250903,190,138,-0.240650,3.054830,0.426316,0.505263
20250904,161,96,-0.041269,3.501254,0.416149,0.559006
20250905,141,91,-0.453744,3.557823,0.468085,0.517730
20250906,86,64,0.017366,3.409370,0.430233,0.558140
20250907,39,29,-0.021465,3.005449,0.435897,0.435897
20250908,137,85,-0.194052,3.139054,0.372263,0.445255
20250909,165,98,-0.474767,3.358944,0.442424,0.496970



[통계 요약]


,n_articles,n_sources,tone_mean,tone_std,tone_neg_share,tone_pos_share
count,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000
mean,132.163934,80.852459,-0.361966,3.431410,0.476594,0.476873
std,40.219059,27.838125,0.770629,0.412252,0.103923,0.097779
min,39.000000,22.000000,-3.089152,2.697976,0.248408,0.218182
25%,122.000000,71.000000,-0.773925,3.139054,0.416149,0.425000
50%,141.000000,85.000000,-0.226114,3.409370,0.454545,0.500000
75%,159.000000,97.000000,0.139593,3.625763,0.525000,0.539877
max,190.000000,140.000000,1.108581,4.978996,0.763636,0.687898


## 3. 테마(Themes) 분리 및 카운팅

In [6]:
def extract_and_count_themes(df, top_n=30):
    """
    themes 컬럼을 분리하고 일별로 각 테마의 등장 횟수를 카운트합니다.
    
    Parameters:
    -----------
    df : pd.DataFrame
        전처리된 기사 데이터
    top_n : int, default=30
        사용할 상위 테마 개수 (전체 기간 기준)
    
    Returns:
    --------
    theme_counts : pd.DataFrame
        일별 테마 카운트 (컬럼명: theme_cnt__THEMENAME)
    
    처리 과정:
    1. themes를 ';' 또는 ','로 분리
    2. explode를 사용해 각 테마를 별도 행으로 분해
    3. 전체 기간에서 상위 N개 테마 선택
    4. 일별, 테마별 카운트 계산
    5. 피벗 테이블로 변환 (행: date, 열: theme)
    """
    print(f"\n{'='*60}")
    print("테마 추출 및 카운팅 시작...")
    print(f"{'='*60}")
    
    # date와 themes 컬럼만 선택
    df_themes = df[['date', 'themes']].copy()
    
    # 1) themes를 ';'와 ','로 분리 (두 구분자 모두 처리)
    # 먼저 ','를 ';'로 통일
    df_themes['themes'] = df_themes['themes'].str.replace(',', ';')
    
    # ';'로 분리하여 리스트로 변환
    df_themes['themes'] = df_themes['themes'].str.split(';')
    print(f"✓ 테마 분리 완료")
    
    # 2) explode를 사용해 각 테마를 별도 행으로 분해
    df_exploded = df_themes.explode('themes')
    print(f"✓ 테마 explode 완료: {len(df_exploded):,}행")
    
    # 빈 문자열이나 공백 제거
    df_exploded['themes'] = df_exploded['themes'].str.strip()
    df_exploded = df_exploded[df_exploded['themes'] != '']
    print(f"✓ 빈 테마 제거 완료: {len(df_exploded):,}행")
    
    # 3) 전체 기간에서 가장 많이 등장한 상위 N개 테마 선택
    theme_total_counts = df_exploded['themes'].value_counts()
    top_themes = theme_total_counts.head(top_n).index.tolist()
    
    print(f"\n✓ 전체 고유 테마 수: {len(theme_total_counts):,}개")
    print(f"✓ 상위 {top_n}개 테마 선택 완료")
    print(f"\n[상위 10개 테마]")
    for i, (theme, count) in enumerate(theme_total_counts.head(10).items(), 1):
        print(f"  {i:2d}. {theme[:50]:50s} : {count:,}회")
    
    # 상위 테마만 필터링
    df_exploded = df_exploded[df_exploded['themes'].isin(top_themes)]
    print(f"\n✓ 상위 {top_n}개 테마로 필터링: {len(df_exploded):,}행")
    
    # 4) 일별 × 테마별 카운트 계산
    theme_counts = df_exploded.groupby(['date', 'themes']).size().reset_index(name='count')
    print(f"✓ 일별 테마 카운트 계산 완료")
    
    # 5) 피벗 테이블로 변환 (행: date, 열: theme)
    theme_pivot = theme_counts.pivot(index='date', columns='themes', values='count')
    
    # 컬럼명에 'theme_cnt__' 접두사 추가
    theme_pivot.columns = ['theme_cnt__' + col for col in theme_pivot.columns]
    
    # 결측치를 0으로 채움 (해당 날짜에 등장하지 않은 테마)
    theme_pivot = theme_pivot.fillna(0).astype(int)
    
    print(f"✓ 피벗 테이블 생성 완료: {len(theme_pivot)}일 × {len(theme_pivot.columns)}개 테마")
    
    return theme_pivot

In [7]:
# 테마 추출 및 카운팅 실행
theme_counts = extract_and_count_themes(df, top_n=30)

# 결과 확인
print("\n[테마 카운트 샘플]")
display(theme_counts.head(10))

print("\n[테마별 통계]")
display(theme_counts.describe().T)


테마 추출 및 카운팅 시작...
✓ 테마 분리 완료
✓ 테마 explode 완료: 284,567행
✓ 빈 테마 제거 완료: 276,505행

✓ 전체 고유 테마 수: 3,404개
✓ 상위 30개 테마 선택 완료

[상위 10개 테마]
   1. ECON_BITCOIN                                       : 7,272회
   2. TAX_FNCACT                                         : 7,135회
   3. EPU_POLICY                                         : 4,651회
   4. WB_696_PUBLIC_SECTOR_MANAGEMENT                    : 3,858회
   5. WB_698_TRADE                                       : 3,853회
   6. EPU_ECONOMY_HISTORIC                               : 3,732회
   7. TAX_ECON_PRICE                                     : 3,658회
   8. USPEC_POLICY1                                      : 3,329회
   9. UNGP_FORESTS_RIVERS_OCEANS                         : 3,155회
  10. TAX_ETHNICITY                                      : 2,601회

✓ 상위 30개 테마로 필터링: 81,449행
✓ 일별 테마 카운트 계산 완료
✓ 피벗 테이블 생성 완료: 61일 × 30개 테마

[테마 카운트 샘플]


,theme_cnt__CRISISLEX_C07_SAFETY,theme_cnt__CRISISLEX_CRISISLEXREC,theme_cnt__ECON_BITCOIN,theme_cnt__ECON_STOCKMARKET,theme_cnt__EPU_CATS_REGULATION,theme_cnt__EPU_ECONOMY,theme_cnt__EPU_ECONOMY_HISTORIC,theme_cnt__EPU_POLICY,theme_cnt__EPU_POLICY_GOVERNMENT,theme_cnt__GENERAL_GOVERNMENT,...,theme_cnt__WB_133_INFORMATION_AND_COMMUNICATION_TECHNOLOGIES,theme_cnt__WB_1920_FINANCIAL_SECTOR_DEVELOPMENT,theme_cnt__WB_2432_FRAGILITY_CONFLICT_AND_VIOLENCE,theme_cnt__WB_507_ENERGY_AND_EXTRACTIVES,theme_cnt__WB_678_DIGITAL_GOVERNMENT,theme_cnt__WB_694_BROADCAST_AND_MEDIA,theme_cnt__WB_696_PUBLIC_SECTOR_MANAGEMENT,theme_cnt__WB_698_TRADE,theme_cnt__WB_831_GOVERNANCE,theme_cnt__WB_840_JUSTICE
date,,,,,,,,,,,,,,,,,,,,,
20250901,15,15,90,20,10,14,35,45,12,13,...,21,11,13,17,21,13,42,52,15,14
20250902,28,19,137,45,31,29,75,66,21,28,...,47,20,25,19,45,39,58,95,33,15
20250903,59,67,183,53,52,42,116,130,49,57,...,62,54,65,30,59,42,120,107,61,44
20250904,39,41,137,48,37,22,72,93,24,31,...,33,32,31,22,30,24,84,85,41,26
20250905,41,31,129,40,20,24,58,85,19,22,...,40,27,43,25,38,23,89,73,45,34
20250906,16,26,72,15,10,32,44,64,12,15,...,13,22,21,11,13,11,35,27,14,19
20250907,10,11,30,8,4,12,17,20,11,12,...,11,7,8,9,11,7,15,17,7,5
20250908,34,45,121,46,41,22,49,72,36,39,...,40,25,20,20,37,16,80,49,38,30
20250909,52,65,152,56,36,35,73,100,21,30,...,44,35,40,19,43,23,96,78,42,48



[테마별 통계]


,count,mean,std,min,25%,50%,75%,max
theme_cnt__CRISISLEX_C07_SAFETY,61.0,34.639344,16.035412,9.0,25.0,33.0,44.0,73.0
theme_cnt__CRISISLEX_CRISISLEXREC,61.0,39.950820,19.699768,11.0,26.0,38.0,49.0,105.0
theme_cnt__ECON_BITCOIN,61.0,119.213115,37.370717,30.0,110.0,129.0,142.0,183.0
theme_cnt__ECON_STOCKMARKET,61.0,34.459016,14.991746,7.0,22.0,36.0,44.0,68.0
theme_cnt__EPU_CATS_REGULATION,61.0,23.704918,11.122566,2.0,16.0,23.0,30.0,52.0
theme_cnt__EPU_ECONOMY,61.0,29.016393,11.808883,8.0,22.0,29.0,35.0,66.0
theme_cnt__EPU_ECONOMY_HISTORIC,61.0,61.180328,23.064047,17.0,49.0,64.0,75.0,119.0
theme_cnt__EPU_POLICY,61.0,76.245902,28.976229,20.0,62.0,77.0,92.0,137.0
theme_cnt__EPU_POLICY_GOVERNMENT,61.0,26.459016,14.087316,4.0,15.0,24.0,33.0,82.0
theme_cnt__GENERAL_GOVERNMENT,61.0,32.016393,16.495951,6.0,19.0,31.0,39.0,89.0


## 4. 최종 데이터 병합 및 저장

In [8]:
def merge_and_save_features(daily_stats, theme_counts, output_path):
    """
    일별 통계와 테마 카운트를 병합하고 CSV 파일로 저장합니다.
    
    Parameters:
    -----------
    daily_stats : pd.DataFrame
        일별 기사량 및 톤 통계
    theme_counts : pd.DataFrame
        일별 테마 카운트
    output_path : str
        출력 파일 경로
    
    Returns:
    --------
    final_df : pd.DataFrame
        병합된 최종 데이터프레임
    
    처리 내용:
    - date를 기준으로 outer join
    - 결측치를 0으로 채움
    - date를 첫 번째 컬럼으로 이동
    - utf-8-sig 인코딩으로 저장
    """
    print(f"\n{'='*60}")
    print("데이터 병합 및 저장 시작...")
    print(f"{'='*60}")
    
    # date를 기준으로 병합 (outer join - 모든 날짜 포함)
    final_df = daily_stats.join(theme_counts, how='outer')
    print(f"✓ 데이터 병합 완료: {len(final_df)}일")
    
    # 결측치를 0으로 채움
    # tone_std는 NaN이 의미가 있을 수 있으므로 0으로 채우지 않음
    fill_cols = ['n_articles', 'n_sources', 'tone_mean', 'tone_neg_share', 'tone_pos_share'] + \
                [col for col in final_df.columns if col.startswith('theme_cnt__')]
    
    for col in fill_cols:
        if col in final_df.columns:
            final_df[col] = final_df[col].fillna(0)
    
    print(f"✓ 결측치 처리 완료")
    
    # date를 인덱스에서 컬럼으로 변환
    final_df = final_df.reset_index()
    
    # 컬럼 순서 정렬: date, 통계 컬럼들, 테마 컬럼들
    stat_cols = ['date', 'n_articles', 'n_sources', 'tone_mean', 'tone_std', 
                 'tone_neg_share', 'tone_pos_share']
    theme_cols = sorted([col for col in final_df.columns if col.startswith('theme_cnt__')])
    final_df = final_df[stat_cols + theme_cols]
    
    print(f"\n[최종 데이터 정보]")
    print(f"  - 행 수: {len(final_df):,}")
    print(f"  - 열 수: {len(final_df.columns):,}")
    print(f"  - 통계 컬럼: {len(stat_cols)}개")
    print(f"  - 테마 컬럼: {len(theme_cols)}개")
    print(f"  - 날짜 범위: {final_df['date'].min()} ~ {final_df['date'].max()}")
    
    # CSV 파일로 저장
    final_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"\n✓ 파일 저장 완료: {output_path}")
    
    return final_df

In [9]:
# 최종 데이터 병합 및 저장 실행
output_file = './data/processed/features_daily.csv'
final_features = merge_and_save_features(daily_stats, theme_counts, output_file)

# 최종 결과 확인
print("\n[최종 Feature 테이블 샘플]")
display(final_features.head(10))

print("\n[데이터 통계 요약]")
display(final_features.describe().T)


데이터 병합 및 저장 시작...
✓ 데이터 병합 완료: 61일
✓ 결측치 처리 완료

[최종 데이터 정보]
  - 행 수: 61
  - 열 수: 37
  - 통계 컬럼: 7개
  - 테마 컬럼: 30개
  - 날짜 범위: 20250901 ~ 20251031

✓ 파일 저장 완료: ./data/processed/features_daily.csv

[최종 Feature 테이블 샘플]


,date,n_articles,n_sources,tone_mean,tone_std,tone_neg_share,tone_pos_share,theme_cnt__CRISISLEX_C07_SAFETY,theme_cnt__CRISISLEX_CRISISLEXREC,theme_cnt__ECON_BITCOIN,...,theme_cnt__WB_133_INFORMATION_AND_COMMUNICATION_TECHNOLOGIES,theme_cnt__WB_1920_FINANCIAL_SECTOR_DEVELOPMENT,theme_cnt__WB_2432_FRAGILITY_CONFLICT_AND_VIOLENCE,theme_cnt__WB_507_ENERGY_AND_EXTRACTIVES,theme_cnt__WB_678_DIGITAL_GOVERNMENT,theme_cnt__WB_694_BROADCAST_AND_MEDIA,theme_cnt__WB_696_PUBLIC_SECTOR_MANAGEMENT,theme_cnt__WB_698_TRADE,theme_cnt__WB_831_GOVERNANCE,theme_cnt__WB_840_JUSTICE
0,20250901,101,50,1.108581,3.068520,0.405941,0.544554,15,15,90,...,21,11,13,17,21,13,42,52,15,14
1,20250902,157,79,0.679788,2.812515,0.248408,0.687898,28,19,137,...,47,20,25,19,45,39,58,95,33,15
2,20250903,190,138,-0.240650,3.054830,0.426316,0.505263,59,67,183,...,62,54,65,30,59,42,120,107,61,44
3,20250904,161,96,-0.041269,3.501254,0.416149,0.559006,39,41,137,...,33,32,31,22,30,24,84,85,41,26
4,20250905,141,91,-0.453744,3.557823,0.468085,0.517730,41,31,129,...,40,27,43,25,38,23,89,73,45,34
5,20250906,86,64,0.017366,3.409370,0.430233,0.558140,16,26,72,...,13,22,21,11,13,11,35,27,14,19
6,20250907,39,29,-0.021465,3.005449,0.435897,0.435897,10,11,30,...,11,7,8,9,11,7,15,17,7,5
7,20250908,137,85,-0.194052,3.139054,0.372263,0.445255,34,45,121,...,40,25,20,20,37,16,80,49,38,30
8,20250909,165,98,-0.474767,3.358944,0.442424,0.496970,52,65,152,...,44,35,40,19,43,23,96,78,42,48
9,20250910,156,80,-0.066913,3.351451,0.435897,0.512821,41,46,145,...,52,33,21,34,43,26,88,75,38,29



[데이터 통계 요약]


,count,mean,std,min,25%,50%,75%,max
n_articles,61.0,132.163934,40.219059,39.000000,122.000000,141.000000,159.000000,190.000000
n_sources,61.0,80.852459,27.838125,22.000000,71.000000,85.000000,97.000000,140.000000
tone_mean,61.0,-0.361966,0.770629,-3.089152,-0.773925,-0.226114,0.139593,1.108581
tone_std,61.0,3.431410,0.412252,2.697976,3.139054,3.409370,3.625763,4.978996
tone_neg_share,61.0,0.476594,0.103923,0.248408,0.416149,0.454545,0.525000,0.763636
tone_pos_share,61.0,0.476873,0.097779,0.218182,0.425000,0.500000,0.539877,0.687898
theme_cnt__CRISISLEX_C07_SAFETY,61.0,34.639344,16.035412,9.000000,25.000000,33.000000,44.000000,73.000000
theme_cnt__CRISISLEX_CRISISLEXREC,61.0,39.950820,19.699768,11.000000,26.000000,38.000000,49.000000,105.000000
theme_cnt__ECON_BITCOIN,61.0,119.213115,37.370717,30.000000,110.000000,129.000000,142.000000,183.000000
theme_cnt__ECON_STOCKMARKET,61.0,34.459016,14.991746,7.000000,22.000000,36.000000,44.000000,68.000000


## 5. 결과 검증

In [10]:
# 생성된 파일 검증
print("\n" + "="*60)
print("결과 검증")
print("="*60)

# 결측치 확인
print("\n[결측치 확인]")
missing_info = final_features.isnull().sum()
missing_info = missing_info[missing_info > 0]
if len(missing_info) > 0:
    print(missing_info)
else:
    print("✓ 결측치 없음 (tone_std 제외)")

# 날짜 연속성 확인
print("\n[날짜 연속성 확인]")
dates = pd.to_datetime(final_features['date'], format='%Y%m%d')
date_diff = dates.diff().dt.days
gaps = date_diff[date_diff > 1]
if len(gaps) > 0:
    print(f"⚠ 날짜 공백 발견: {len(gaps)}개")
    print(gaps)
else:
    print("✓ 날짜 연속적")

# 기사 수 분포
print("\n[일별 기사 수 분포]")
print(f"  - 최소: {final_features['n_articles'].min():.0f}")
print(f"  - 최대: {final_features['n_articles'].max():.0f}")
print(f"  - 평균: {final_features['n_articles'].mean():.1f}")
print(f"  - 중앙값: {final_features['n_articles'].median():.0f}")

# tone 분포
print("\n[Tone 분포]")
print(f"  - 평균 tone: {final_features['tone_mean'].mean():.3f}")
print(f"  - 평균 부정 비율: {final_features['tone_neg_share'].mean():.1%}")
print(f"  - 평균 긍정 비율: {final_features['tone_pos_share'].mean():.1%}")

# 상위 테마 활동도
print("\n[가장 활발한 상위 5개 테마]")
theme_cols = [col for col in final_features.columns if col.startswith('theme_cnt__')]
theme_totals = final_features[theme_cols].sum().sort_values(ascending=False)
for i, (theme, count) in enumerate(theme_totals.head(5).items(), 1):
    theme_name = theme.replace('theme_cnt__', '')
    avg_per_day = count / len(final_features)
    print(f"  {i}. {theme_name[:40]:40s} : 총 {count:.0f}회 (일평균 {avg_per_day:.1f}회)")

print("\n" + "="*60)
print("✓ 모든 작업 완료!")
print("="*60)


결과 검증

[결측치 확인]
✓ 결측치 없음 (tone_std 제외)

[날짜 연속성 확인]
✓ 날짜 연속적

[일별 기사 수 분포]
  - 최소: 39
  - 최대: 190
  - 평균: 132.2
  - 중앙값: 141

[Tone 분포]
  - 평균 tone: -0.362
  - 평균 부정 비율: 47.7%
  - 평균 긍정 비율: 47.7%

[가장 활발한 상위 5개 테마]
  1. ECON_BITCOIN                             : 총 7272회 (일평균 119.2회)
  2. TAX_FNCACT                               : 총 7135회 (일평균 117.0회)
  3. EPU_POLICY                               : 총 4651회 (일평균 76.2회)
  4. WB_696_PUBLIC_SECTOR_MANAGEMENT          : 총 3858회 (일평균 63.2회)
  5. WB_698_TRADE                             : 총 3853회 (일평균 63.2회)

✓ 모든 작업 완료!
